In [3]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

# 1. Definisikan arsitektur model persis sama dengan saat training.
class IndoBERT_BiLSTM(nn.Module):
    def __init__(self, bert_model="indobenchmark/indobert-base-p1", lstm_hidden=128, num_classes=3):
        super(IndoBERT_BiLSTM, self).__init__()
        
        # Load IndoBERT sebagai feature extractor.
        self.bert = AutoModel.from_pretrained(bert_model)
        self.bert.requires_grad_(False)  # Freeze bobot IndoBERT agar tidak berubah saat inferensi.
        
        # BiLSTM untuk memproses representasi vektor dari IndoBERT.
        self.lstm = nn.LSTM(
            input_size=768, 
            hidden_size=lstm_hidden, 
            num_layers=2,
            batch_first=True, 
            bidirectional=True, 
            dropout=0.3
        )
        
        # Batch Normalization dan Dropout untuk membantu stabilitas.
        self.batch_norm = nn.BatchNorm1d(lstm_hidden * 2)
        self.dropout = nn.Dropout(0.3)
        
        # Fully Connected layer untuk menghasilkan prediksi kelas.
        self.fc = nn.Linear(lstm_hidden * 2, num_classes)
        
    def forward(self, input_ids, attention_mask):
        # Ekstraksi fitur dengan IndoBERT.
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Mengambil representasi token [CLS] (index 0) sebagai representasi kalimat.
        bert_embedding = bert_output.last_hidden_state[:, 0, :]  # Bentuk: [batch_size, 768]
        
        # Proses dengan BiLSTM. Karena LSTM mengharapkan input 3 dimensi, kita tambahkan dimensi sekuens.
        lstm_out, _ = self.lstm(bert_embedding.unsqueeze(1))  # Bentuk: [batch_size, seq_len=1, hidden_size*2]
        lstm_out = lstm_out[:, -1, :]  # Ambil output terakhir dari LSTM.
        
        # Normalisasi dan dropout.
        lstm_out = self.batch_norm(lstm_out)
        lstm_out = self.dropout(lstm_out)
        
        # Prediksi akhir.
        output = self.fc(lstm_out)
        return output

# 2. Inisialisasi model dan device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = IndoBERT_BiLSTM().to(device)

# 3. Load bobot model yang telah disimpan.
# Pastikan file "best_model.pth" berada di direktori kerja Anda.
model.load_state_dict(torch.load("./model/best_model.pth", map_location=device))
model.eval()  # Set model ke mode evaluasi.

# 4. Load tokenizer yang sama dengan saat training.
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# 5. Buat mapping label.
# Misalnya: {0: "negatif", 1: "netral", 2: "positif"}
label_mapping = {0: "negatif", 1: "netral", 2: "positif"}

# 6. Fungsi prediksi.
def predict(text, model, tokenizer, device, label_mapping):
    """
    Fungsi untuk melakukan prediksi sentimen pada input teks.
    Args:
        text (str): Komentar atau kalimat input.
        model: Model yang telah diload.
        tokenizer: Tokenizer IndoBERT.
        device: Device yang digunakan (CPU atau GPU).
        label_mapping (dict): Mapping indeks ke label (misal: 0->"negatif").
    Returns:
        str: Label sentimen prediksi.
    """
    model.eval()  # Pastikan model dalam mode evaluasi.
    
    # Tokenisasi teks input.
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Lakukan inferensi tanpa gradien.
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
    
    # Ambil indeks prediksi dengan nilai tertinggi.
    pred_idx = torch.argmax(outputs, dim=1).item()
    return label_mapping[pred_idx]

# 7. Contoh penggunaan prediksi.
example_text = "Produk ini sangat bagus dan berkualitas!"
prediction = predict(example_text, model, tokenizer, device, label_mapping)
print(f"Prediksi Sentimen: {prediction}")

Prediksi Sentimen: positif


In [2]:
example_text = "Produk ini sangat jelek"
prediction = predict(example_text, model, tokenizer, device, label_mapping)
print(f"Prediksi Sentimen: {prediction}")

Prediksi Sentimen: negatif


In [7]:
example_text = "aku udahh Beli nantii aku Komen lagi Kalo ada perubahan"
prediction = predict(example_text, model, tokenizer, device, label_mapping)
print(f"Prediksi Sentimen: {prediction}")

Prediksi Sentimen: negatif


In [4]:
import pandas as pd

train_df = pd.read_csv("./test_comments_with_predictions.csv")

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer

# 1. Load dataset CSV yang berisi komentar
test_df = pd.read_csv("./dataset/comments.csv")  # Ganti dengan path file Anda

# Pastikan kolom "comment" ada di dataset
if "comment" not in test_df.columns:
    raise ValueError("Kolom 'comment' tidak ditemukan dalam dataset!")

# 2. Bersihkan dataset (pastikan semua komentar bertipe string)
test_df["comment"] = test_df["comment"].astype(str)  # Konversi semua ke string
test_df["comment"] = test_df["comment"].fillna("")   # Jika ada NaN, ganti dengan string kosong

# 3. Definisikan ulang fungsi prediksi dengan validasi input
def predict(text, model, tokenizer):
    if not isinstance(text, str) or text.strip() == "":
        return "netral"  # Label default jika komentar kosong atau tidak valid

    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        output = model(input_ids, attention_mask)
    
    pred_class = output.argmax(dim=1).item()
    
    mapping = {0: "negatif", 1: "netral", 2: "positif"}
    return mapping[pred_class]

# 4. Lakukan prediksi untuk setiap komentar di dataset
predicted_labels = []
for comment in test_df["comment"]:
    pred_label = predict(comment, model, tokenizer)
    predicted_labels.append(pred_label)

# 5. Tambahkan hasil prediksi ke dataset
test_df["predicted_label"] = predicted_labels

# 6. Simpan hasil prediksi ke file CSV baru
output_path = "test_comments_with_predictions.csv"
test_df.to_csv(output_path, index=False)
print(f"Hasil prediksi telah disimpan ke {output_path}")


NameError: name 'model' is not defined

In [6]:
train_df.head()

,no,comment,predicted_label
0,1,"Aku cuma pake sunscreen dan krim ini doang, al...",positif
1,2,"Simpen dulu di keranjang kuning, nanti check out.",negatif
2,3,Muka kusamku jadi segeran setelah rutin pake k...,positif
3,4,"Aku cuma pake sunscreen dan krim ini doang, al...",positif
4,5,Masih menjadi misteri kenapa Dr. Fay bisa seba...,negatif


In [7]:
import re

# 1. Siapkan kamus kata sentimen
positive_words = {"bagus", "baik", "indah", "cerah", "puas", "suka", "alhamdulillah"}
negative_words = {"buruk", "jelek", "kecewa", "mengecewakan"}
negation_words = {"tidak", "bukan", "kurang", "belum"}

def preprocess_negation(text):
    """
    - Ubah ke huruf kecil dan ekstrak hanya kata (tanpa tanda baca)
    - Jika menemukan kata negasi dan kata berikutnya ada dalam positive atau negative words,
      maka tandai kata berikutnya dengan prefix "NEG_" (untuk positive) atau "POS_" (untuk negative)
    - Jika tidak, proses kata seperti biasa
    """
    # Ekstrak kata dengan regex agar tanda baca tidak ikut
    words = re.findall(r'\b\w+\b', text.lower())
    new_words = []
    i = 0
    while i < len(words):
        word = words[i]
        # Jika kata merupakan negasi dan ada kata setelahnya
        if word in negation_words and i + 1 < len(words):
            next_word = words[i+1]
            # Jika kata berikutnya termasuk kata positif, tandai dengan "NEG_"
            if next_word in positive_words:
                new_words.append("NEG_" + next_word)
                i += 2  # Lewati kata negasi dan kata yang ditandai
                continue
            # Jika kata berikutnya termasuk kata negatif, tandai dengan "POS_"
            elif next_word in negative_words:
                new_words.append("POS_" + next_word)
                i += 2
                continue
            else:
                # Jika kata berikutnya bukan kata sentimen, simpan kata negasi dan lanjutkan
                new_words.append(word)
        else:
            new_words.append(word)
        i += 1
    return " ".join(new_words)

def sentiment_score(text):
    """
    Fungsi ini menghitung skor sentimen berdasarkan:
      - Menambahkan +1 untuk setiap kata positif
      - Mengurangi -1 untuk setiap kata negatif
      - Untuk kata yang diberi prefix "NEG_", mengurangi -1 (karena kata positif yang dinetralkan)
      - Untuk kata dengan prefix "POS_", menambahkan +1 (karena kata negatif yang dibalik artinya)
    """
    processed_text = preprocess_negation(text)
    score = 0
    words = processed_text.split()
    for word in words:
        if word in positive_words:
            score += 1
        elif word in negative_words:
            score -= 1
        elif word.startswith("NEG_"):
            # Kata positif yang di-negate dihitung sebagai negatif
            if word[4:] in positive_words:
                score -= 1
        elif word.startswith("POS_"):
            # Kata negatif yang di-negate dihitung sebagai positif
            if word[4:] in negative_words:
                score += 1
    if score > 0:
        return "positif"
    elif score < 0:
        return "negatif"
    else:
        return "netral"

# Contoh pengujian:
texts = [
    "Bagus sekali, saya suka!",         # Seharusnya: positif
    "Tidak bagus sama sekali",            # Seharusnya: negatif
    "Biasa saja tidak terlalu buruk atau bagus"  # Seharusnya: netral
]

for text in texts:
    print(f"{text} -> {sentiment_score(text)}")


Bagus sekali, saya suka! -> positif
Tidak bagus sama sekali -> negatif
Biasa saja tidak terlalu buruk atau bagus -> netral
